In [1]:
from datasets import PACSDataset
from torchvision import transforms
from torch.utils.data import DataLoader


# Training a neural network
We are able to load data from our custom datasets, now we need to train a neural network. We need the following:

1. A neural network
2. An optimiser
3. A training loop that iterates through samples provided by a dataloader and uses the optimiser to update the neural networks weights


## Train and Test dataloaders
Normally, we'll have a training, validation and test set. The validation set is used for hyperparameter tuning. Since we won't do any hyperparameter tuning we can just go ahead and use the test set for evaluation. 

In [2]:
# Files and transforms
with open("train_files.txt") as f:
    file_names_train = f.read().splitlines()
    
with open("test_files.txt") as f:
    file_names_test = f.read().splitlines()

# transforms
# A great point to add data augmentations - you want to do class-preserving transformations
# Rotations, Reflections etc
transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        ])
train_dataset = PACSDataset(file_names_train, transform)
test_dataset = PACSDataset(file_names_test, transform)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

## Neural network modules
We will create a neural network modules. To illustrate the modularity of pytorch we will break it down into multiple modules.

1. A featurizer: applies convolutions to extract "useful" features
2. A classifier: fully connected net built on top of featurizer
3. A network that combines the two: Allows us to easily swap out parts 1 or 2. 

In [3]:
from torch import nn
from torch.nn import functional as F
import torch


# Featurizer
class Featurizer(nn.Module):
    def __init__(self):
        super(Featurizer, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.dropout = nn.Dropout2d(0.5)
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return x

# Classifier
# We will hardcode the layers for now, but it is best to use parameters
class Classifier(nn.Module):
    def __init__(self, num_classes: int = 7):
        super(Classifier, self).__init__()
        # make a point here about the 12544
        self.fc1 = nn.Linear(12544, 4096)
        self.dropout = nn.Dropout2d(0.5)

        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Combination
class CNNClassifier(nn.Module):
    def __init__(self, num_classes: int = 7):
        super(CNNClassifier, self).__init__()
        self.featurizer = Featurizer()
        self.classifier = Classifier(num_classes)

    def forward(self, x):
        return self.classifier(self.featurizer(x))


In [4]:
# Initialise network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNNClassifier(num_classes=7)
model.to(device)
print(model)

CNNClassifier(
  (featurizer): Featurizer(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (dropout): Dropout2d(p=0.5, inplace=False)
    (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Classifier(
    (fc1): Linear(in_features=12544, out_features=4096, bias=True)
    (dropout): Dropout2d(p=0.5, inplace=False)
    (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    (fc3): Linear(in_features=1024, out_features=7, bias=True)
  )
)


## Training logic
As mentioned before, we can fit the training logic into a class with a network as an attribute.

We also need to consider hyperparameters

In [5]:
learning_rate = 1e-4
# batch_size = 64
epochs = 16

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    if (t+1) % 5 == 0:
        test_loop(train_dataloader, model, loss_fn)
        test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


C:\Users\Shane Josias\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 1.943914  [    0/ 6993]
loss: 1.948635  [ 1280/ 6993]
loss: 1.943341  [ 2560/ 6993]
loss: 1.948219  [ 3840/ 6993]
loss: 1.939206  [ 5120/ 6993]
loss: 1.950150  [ 6400/ 6993]
Epoch 2
-------------------------------
loss: 1.945499  [    0/ 6993]
loss: 1.941038  [ 1280/ 6993]
loss: 1.928608  [ 2560/ 6993]
loss: 1.944826  [ 3840/ 6993]
loss: 1.933159  [ 5120/ 6993]
loss: 1.936315  [ 6400/ 6993]
Epoch 3
-------------------------------
loss: 1.941919  [    0/ 6993]
loss: 1.951895  [ 1280/ 6993]
loss: 1.946172  [ 2560/ 6993]
loss: 1.905301  [ 3840/ 6993]
loss: 1.932095  [ 5120/ 6993]
loss: 1.937509  [ 6400/ 6993]
Epoch 4
-------------------------------
loss: 1.923451  [    0/ 6993]
loss: 1.941015  [ 1280/ 6993]
loss: 1.906628  [ 2560/ 6993]
loss: 1.925535  [ 3840/ 6993]
loss: 1.932081  [ 5120/ 6993]
loss: 1.921762  [ 6400/ 6993]
Epoch 5
-------------------------------
loss: 1.916324  [    0/ 6993]
loss: 1.885046  [ 1280/ 6993]
loss: 1.937992  [ 2560/ 6993]
loss: 1.918097  [ 3840/ 6993]
